# Analysis

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import utils
import matplotlib.pyplot as plt
import yt
from rich import print
from icecream import ic
import panel as pn
from loguru import logger

import logging
import json
from yt.data_objects.time_series import SimulationTimeSeries
from yt.data_objects.static_output import Dataset
from matplotlib.pyplot import Axes

In [4]:
dim = 1
# dim = 2
# dim = 3
beta = 0.25
theta = 60.0
plasma_resistivity = 100.0

In [5]:
import os
from pathlib import Path

try:
    base_dir = Path(os.getcwd()) / "01_oblique_linear_alfven"
    sub_dir = f"dim_{dim}_beta_{beta}_theta_{theta}_eta_{plasma_resistivity}"
    directory = base_dir / sub_dir
    os.chdir(directory)
    os.makedirs("figures", exist_ok=True)
except FileNotFoundError:
    pass

In [6]:
pn.extension()

# load simulation parameters
with open("sim_parameters.json", "rb") as f:
    meta = json.load(f)

In [7]:
ps = "ions"

In [8]:
import numpy as np
import unyt as u
from utils import check_ds_type
from unyt.physical_constants import mu_0

def add_V_Alfven_field(ds: Dataset, direction: str):
    #: NOTE: rho is species charge density

    def _V_Alfven(field, data):
        rho_m = data["rho"] * meta["m_ion"] * u.m**(-3) / u.qp_mks.value * u.kg
        return data[f"B{direction}"] / np.sqrt(
            mu_0 * rho_m
        )

    name = ("boxlib", f"V_Alfven_{direction}")
    
    ds.add_field(name, function=_V_Alfven, units="km/s", sampling_type="local")


def add_V_field(ds: Dataset, direction: str):
    def _V(field, data):
        return data[f"particle_momentum_{direction}"] / meta["m_ion"] / u.kg

    name = (ps, f"V_{direction}")

    ds.add_field(name, function=_V, units="km/s", sampling_type="particle")


def add_field(ds: Dataset):
    type = check_ds_type(ds)
    if type == "particle":
        add_V_field(ds, "x")
        add_V_field(ds, "y")
        add_V_field(ds, "z")
    elif type == "field":
        add_V_Alfven_field(ds, "x")
        add_V_Alfven_field(ds, "y")
        add_V_Alfven_field(ds, "z")

## Fields

In [9]:
from utils import load_ts_all
ts_field, ts_part = load_ts_all(meta) 

In [10]:
ic(len(ts_field))
ds_field: Dataset = ts_field[0]
ds_part: Dataset = ts_part[0]

ic| len(ts_field): 501
yt : [INFO     ] 2024-05-01 16:15:55,789 Parameters: current_time              = 0.0
yt : [INFO     ] 2024-05-01 16:15:55,790 Parameters: domain_dimensions         = [256   1   1]
yt : [INFO     ] 2024-05-01 16:15:55,790 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-05-01 16:15:55,790 Parameters: domain_right_edge         = [2.91469729e+06 1.00000000e+00 1.00000000e+00]
yt : [INFO     ] 2024-05-01 16:15:55,813 Parameters: current_time              = 0.0
yt : [INFO     ] 2024-05-01 16:15:55,813 Parameters: domain_dimensions         = [256   1   1]
yt : [INFO     ] 2024-05-01 16:15:55,814 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-05-01 16:15:55,814 Parameters: domain_right_edge         = [2.91469729e+06 1.00000000e+00 1.00000000e+00]


## Particles

In [11]:
def test_plot_part():

    x_field = f"particle_position_{direction}"
    y_field = "particle_momentum_y"

    p = yt.ParticlePlot(
        ds,
        x_field,
        y_field,
        (ps, "particle_weight"),
    )
    p.set_log(y_field, False)
    return p

# test_plot_part()

In [12]:
# profile = create_part_profile(
    # ds_part.all_data(),
    # fields=[weight_field, field],
# )

In [13]:
# plot_field_with_plasma_profile_ts(ts_field, ts_part, name="v_comp", step=4)

In [14]:
#: check the field list
# print(ds_part.field_list)
# print(ds_part.derived_field_list)

In [15]:
def test_add_field():
    fname = ds.add_deposited_particle_field(
        (ps, "particle_momentum_y"),
        method="nearest"
    )
    print(fname)
    print(ds.fluid_types)
    print(ds.particle_types)
    return fname


# YTFieldTypeNotFound: Could not find field type 'deposit'.

### Pressure

In [16]:
import polars as pl
import polars.selectors as cs
import numpy as np

In [17]:
# df["x"] = pd.qcut(df["particle_position_x"], q=10)

In [22]:
from utils import ds2df

n = 64
if dim == 1 or dim == 2:
    i = 0
else:
    i = 2

def calc_pressure(ds, direction):
    df =  pl.DataFrame(ds2df(ds))
    breaks = np.linspace(ds.domain_left_edge[i], ds.domain_right_edge[i], ds.domain_dimensions[i]+1)
    col = f"particle_position_{direction}"
    return df.with_columns(
        # pl.col(f"particle_position_{direction}").qcut(n).alias(direction),
        pl.col(col).cut(breaks, include_breaks=True),
    ).with_columns(
        (cs.contains("momentum") - cs.contains("momentum").mean().over(col))
        .pow(2)
        .name.suffix("_var"),
        # cs.contains("momentum").mean().over(direction).name.suffix("_mean"),
    ).group_by(col).agg(
        cs.contains("var").sum(),
        pl.col("time").first()
    ).unnest(col).rename({"brk": direction}).drop(cs.by_dtype(pl.Categorical))

In [20]:
from utils import ds2xr, ds2df

In [21]:
ds_field

WarpXDataset: /Users/zijin/projects/swd_simulation/warpx/01_oblique_linear_alfven/dim_1_beta_0.25_theta_60.0_eta_100.0/diags/diag1000000

In [47]:
Bfields = ["Bx", "By", "Bz"]

p_df = pl.concat(calc_pressure(ds, direction) for ds in ts_part).with_columns(pl.col(direction).rank("dense"))
b_df = pl.concat(pl.DataFrame(ds2df(ds, fields=Bfields)) for ds in ts_field).with_columns(pl.col(direction).rank("dense"))
# df_part.write_ipc("part_pressure.arrow")

In [51]:
p_df = p_df.rename(
    {
        "particle_momentum_x_var": "p_xx",
        "particle_momentum_y_var": "p_yy",
        "particle_momentum_z_var": "p_zz",
    }
)

b_df = b_df.with_columns(
    Bmag = np.linalg.norm(b_df[Bfields],axis=1)
)

In [52]:
direction

'z'

In [53]:
direction = "z"

df = p_df.join(
    b_df, on=["time", direction]
).with_columns(
    p_parp=(
        pl.col("p_xx") * pl.col("Bx").abs()
        + pl.col("p_yy") * pl.col("By").abs()
        + pl.col("p_zz") * pl.col("Bz").abs()
    )
    / pl.col("Bmag")
).with_columns(
    p_perp = (pl.col("p_xx") + pl.col("p_yy") + pl.col("p_zz") - pl.col("p_parp")) / 2
).with_columns(
    anisotropy = pl.col("p_perp") / pl.col("p_parp")
)
df.write_ipc("pressure.arrow")

In [54]:
df

z,p_xx,p_yy,p_zz,time,Bx,By,Bz,x,y,Bmag,p_parp,p_perp,anisotropy
u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1.0541e-42,8.2167e-43,1.0052e-42,0.0,8.6603e-8,9.9970e-8,5.0000e-8,0.5,0.5,1.4140e-7,1.5820e-42,6.4951e-43,0.410571
2,1.3321e-42,1.1860e-42,6.6635e-43,0.0,8.6603e-8,9.9729e-8,5.0000e-8,0.5,0.5,1.4123e-7,1.8902e-42,6.4709e-43,0.342339
3,1.0122e-42,7.5734e-43,1.1687e-42,0.0,8.6603e-8,9.9248e-8,5.0000e-8,0.5,0.5,1.4089e-7,1.5704e-42,6.8391e-43,0.435488
4,8.2195e-43,1.1861e-42,1.0771e-42,0.0,8.6603e-8,9.8528e-8,5.0000e-8,0.5,0.5,1.4038e-7,1.7231e-42,6.8101e-43,0.395217
5,8.4323e-43,1.0714e-42,1.1372e-42,0.0,8.6603e-8,9.7570e-8,5.0000e-8,0.5,0.5,1.3971e-7,1.6778e-42,6.8696e-43,0.409429
…,…,…,…,…,…,…,…,…,…,…,…,…,…
252,1.8606e-42,1.4379e-42,1.0721e-42,327.972254,9.0782e-8,3.1514e-8,5.0000e-8,0.5,0.5,1.0833e-7,2.4724e-42,9.4910e-43,0.383871
253,2.0428e-42,1.1462e-42,2.0095e-42,327.972254,9.1139e-8,3.2258e-8,5.0000e-8,0.5,0.5,1.0884e-7,2.9734e-42,1.1126e-42,0.374187
254,1.7203e-42,1.1514e-42,1.1030e-42,327.972254,8.7394e-8,3.1365e-8,5.0000e-8,0.5,0.5,1.0546e-7,2.2911e-42,8.4186e-43,0.367452


In [102]:
pl.concat([b_df, p_df.drop("time")], how="horizontal")

Bx,By,Bz,Ex,Ey,Ez,jx,jy,jz,rho,z,x,y,time,particle_position_z,particle_momentum_x_var,particle_momentum_y_var,particle_momentum_z_var
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
8.6603e-8,9.9970e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.5826e-11,5692.768137,0.5,0.5,0.0,11385.536275,1.0541e-42,8.2167e-43,1.0052e-42
8.6603e-8,9.9729e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.5881e-11,17078.304412,0.5,0.5,0.0,22771.072549,1.3321e-42,1.1860e-42,6.6635e-43
8.6603e-8,9.9248e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.5946e-11,28463.840686,0.5,0.5,0.0,34156.608824,1.0122e-42,7.5734e-43,1.1687e-42
8.6603e-8,9.8528e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.6274e-11,39849.376961,0.5,0.5,0.0,45542.145098,8.2195e-43,1.1861e-42,1.0771e-42
8.6603e-8,9.7570e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.6521e-11,51234.913235,0.5,0.5,0.0,56927.681373,8.4323e-43,1.0714e-42,1.1372e-42
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8.6603e-8,9.7570e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.6094e-11,2.8635e6,0.5,0.5,0.0,2.8692e6,1.0439e-42,1.3101e-42,1.0987e-42
8.6603e-8,9.8528e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.5885e-11,2.8748e6,0.5,0.5,0.0,2.8805e6,9.6623e-43,1.1275e-42,1.1714e-42
8.6603e-8,9.9248e-8,5.0000e-8,0.0,0.0,0.0,0.0,0.0,0.0,1.6237e-11,2.8862e6,0.5,0.5,0.0,2.8919e6,8.1829e-43,9.9498e-43,1.2100e-42


In [ ]:
ds.domain_dimensions

In [ ]:
df_part

In [ ]:
df_part.plot(x="z", y="time")

### yt

In [ ]:
ts: SimulationTimeSeries = yt.load('diags/diag1??????')
# ts = yt.load('./diags/diag???0032')

In [ ]:
def plot(ds, normalize = True, **kwargs):
    ad = ds.all_data()
    fields = ["Bx", "By", "Bz"]

    match meta["dim"]:
        case 1: pos = "x"
        case 2: pos = "y"
        
    pos = ad[pos]
    
    if normalize:
        pos = pos / meta['d_i']
    
    for field in fields:
        plt.plot(pos, ad[field], label=field, **kwargs)
    
    plt.xlabel("x ($d_i$)")
        
def hodogram(ds, comp1="By", comp2="Bz"):
    time = ds.current_time
    time_norm = time.value / meta['t_ci']
    ad = ds.all_data()
    plt.plot(ad[comp1], ad[comp2], label=f"t={time_norm:.2f}")
    plt.xlabel(comp1)
    plt.ylabel(comp2)

In [ ]:
for i, ds in enumerate(ts):
    alpha = (i + 1) / (len(ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
i = 4
_ts = ts[0:i]
for i, ds in enumerate(_ts):
    alpha = (i + 1) / (len(_ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
for ds in ts:
    hodogram(ds)
    plt.legend()

In [ ]:
i = 5
for ds in ts[0:i]:
    hodogram(ds)
    plt.legend()

In [ ]:
yt.SlicePlot(ds, "z", ("boxlib", "Bz"))

In [ ]:
ds.all_data()

In [ ]:
fields = [
    ("Bx"),
    ("By"),
    ("Bz"),
    ("mesh", "magnetic_field_strength"),
]

In [ ]:
for ds in ts.piter():
    p = yt.plot_2d(ds, fields=fields)
    p.set_log(fields, False)
    fig = p.export_to_mpl_figure((2, 2))
    fig.tight_layout()
    fig.savefig(f"figures/{ds}_magnetic_field.png")

### Average magnetic field

In [ ]:
def plot_avg(ds):
    fields = [
        ("Bx"),
        ("By"),
        ("Bz"),
    ]

    ad = ds.all_data()
    df = ad.to_dataframe(fields + ["x", zaxis])
    # compute the magnetic field strength
    df = df.assign(B=lambda x: (x.Bx**2 + x.By**2 + x.Bz**2) ** 0.5)

    axes = df.groupby(zaxis).mean().plot(y=fields + ["B"], subplots=True)
    return axes[0].figure

In [ ]:
def plot_avg_ts(i):
    return plot_avg(ts[i])
    
time_widget = pn.widgets.IntSlider(name="Time", value=1, start=0, end=len(ts)-1)
bound_plot = pn.bind(plot_avg_ts, i=time_widget)

pn.Column(time_widget, bound_plot)

In [ ]:
for ds in ts.piter():
    plot_avg(ds)

In [ ]:
ds.print_stats()
print(ds.field_list)

In [ ]:
grid = ds.r[:,:,:]
obj = grid.to_xarray(fields=fields)